In [1]:
import cv2
import numpy as np
import json
import os
from tqdm import tqdm

# Define the path to the folder containing the masks
mask_folder = 'C:\\Users\\pjrio\\Mask_RCNN\\dataset\\masks'
# Load all of the masks in the folder
mask_files = [f for f in os.listdir(mask_folder) if f.endswith('.jpg')]

# Initialize the COCO format dataset
coco_output = {}

image_id = 0

print("--Processing images:")
# Iterate over all of the masks
for i, mask_file in tqdm(enumerate(mask_files), total=len(mask_files)):
    # Get the image name without extension
    image_name = os.path.splitext(mask_file)[0]
    
    # Iterate over all of the masks, Load the mask image and convert it to binary format, Find the contours of the objects in the binary mask
    mask = cv2.imread(os.path.join(mask_folder, mask_file), cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (256, 256))

    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    
    # Find the contours of the objects in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    region_id = 0
    regions = {}
    for j, contour in enumerate(contours):
        contour = contour.flatten().tolist()
        x = [point for point in contour[0::2]]
        y = [point for point in contour[1::2]]
        if len(x) < 2:
            continue
        region = {
            "region_attributes": {},
            "shape_attributes": {
                "all_points_x": x,
                "all_points_y": y,
                "name": "polygon"
            }
        }
        regions[str(region_id)] = region
        region_id += 1
     # Get the base file name without the "_label" suffix
    base_filename, _ = os.path.splitext(mask_file)
    base_filename = base_filename.replace("_label", "")
    
    coco_output[image_name + mask_file] = {
        "fileref": "",
        "size": os.path.getsize(os.path.join(mask_folder, mask_file)),
        "filename": f"{base_filename}.jpg",
        "base64_img_data": "",
        "file_attributes": {},
        "regions": regions
    }

# Save the COCO format dataset to a JSON file
print("Exporting to annotations-all.json.")
with open('annotations-all.json', 'w') as fp:
    json.dump(coco_output, fp, indent=4, separators=(',', ': '))

--Processing images:


100%|██████████| 3217/3217 [00:06<00:00, 509.57it/s]


Exporting to annotations-all.json.
